### To setup the environment for downloading the dataset and the YOLOv5 model.

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt
!pip install roboflow

import torch

from roboflow import Roboflow
rf = Roboflow(api_key="RtdzCZNW8mDA4ilAU416")
project = rf.workspace("projects-1vokb").project("frogs-ft5yx")
dataset = project.version(1).download("yolov5")

### Train on polygons

In [ ]:
!python yolov5/segment/train.py --data frogs-1/data.yaml --img 320 --epochs 10000 --batch 128 --name trained-seg

### Test model on bounding boxes

In [ ]:
!python yolov5/segment/predict.py --save-txt --save-conf --weights yolov5/train-seg/trained-seg/weights/best.pt --img 320  --source 'frogs-1/test/images/*.jpg'

**Convert both the predicted polygons and the polygon test set from polygon to bounding box coordinates.**

In [ ]:
import numpy as np
import os
from sklearn.metrics import pairwise_distances
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from sklearn.metrics import precision_recall_curve
from torchmetrics.classification import PrecisionRecallCurve
import cv2

def convert_to_bounding_box(segmentation_label):
    x_coord, y_coord = [], []
    for i in range(0, len(segmentation_label), 2):
        x_coord.append(segmentation_label[i])
        y_coord.append(segmentation_label[i+1])
    return convert_to_yolov5(min(x_coord), min(y_coord), max(x_coord), max(y_coord))

def convert_to_yolov5(xmin, ymin, xmax, ymax):
    x_center = (xmin + xmax) / 2
    y_center = (ymin + ymax) / 2
    width = xmax - xmin
    height = ymax - ymin
    return np.array((x_center, y_center, width, height))


def from_xmin_ymin_xmax_ymax_to_yolov5(folder_path, confidence=True):
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as file:
                lines = file.readlines()
            idx_to_delete = set()
            for i, line in enumerate(lines):
                values = line.split()
                if len(values) < 6:
                    idx_to_delete.add(i)
                    continue
                class_label = values[0]
                if confidence:
                    conf = values[-1]
                    polygon = [float(val) for val in values[1:-1]]
                    bbox = convert_to_bounding_box(polygon)
                    lines[i] = ' '.join(map(str, bbox)) + str(" ") + str(conf) + str(" ") + str(class_label) + "\n"
                else:
                    polygon = [float(val) for val in values[1:]]
                    bbox = convert_to_bounding_box(polygon)
                    lines[i] = f"{class_label} {' '.join(map(str, bbox))}"  + "\n"
            new_lines = []
            for idx, line in enumerate(lines):
                if idx not in idx_to_delete:
                    new_lines.append(line)
            with open(file_path, 'w') as file:
                file.writelines(new_lines)

In [ ]:
preds_folder = "yolov5/runs/predict-seg/exp/labels/" # predicted polygons
gt_folder = "frogs-1/test/labels/" # ground truth polygons

from_xmin_ymin_xmax_ymax_to_yolov5(preds_folder, confidence=True)
from_xmin_ymin_xmax_ymax_to_yolov5(gt_folder, confidence=False)

### Compute metrics
**Run this cell tu get the Precision, Recall and mAP**

In [ ]:
import os

# Set parameters for computing AP
confidence_threshold = 0
iou_threshold = 0

# Initialize variables for computing precision, recall, and AP
true_positives = []
false_positives = []
total_gt_objects = 0
c = 0

# Loop through each file in gt folder
for file_name in os.listdir(gt_folder):
    c += 1
    # Load ground truth data
    gt_file_path = os.path.join(gt_folder, file_name)
    with open(gt_file_path, "r") as f:
        gt_data = f.readlines()
    gt_data = [line.strip().split() for line in gt_data]

    # Load prediction data
    preds_file_path = os.path.join(preds_folder, file_name)
    with open(preds_file_path, "r") as f:
        preds_data = f.readlines()
    preds_data = [line.strip().split() for line in preds_data]

    # Initialize variables for computing recall and AP for this file
    num_gt_objects = len(gt_data)
    total_gt_objects += num_gt_objects
    tp = [False] * num_gt_objects
    fp = [False] * len(preds_data)
    
    # Loop through each prediction
    for pred_idx, pred in enumerate(preds_data):
        pred_x, pred_y, pred_w, pred_h, pred_conf, pred_class = pred
        if pred_class == '-1':
            pass
        pred_x, pred_y, pred_w, pred_h, pred_conf = map(float, [pred_x, pred_y, pred_w, pred_h, pred_conf])
        pred_conf = float(pred_conf)

        # Find best matching ground truth object
        best_iou = 0
        best_gt_idx = -1
        for gt_idx, gt in enumerate(gt_data):
            gt_class, gt_x, gt_y, gt_w, gt_h = gt
            gt_x, gt_y, gt_w, gt_h = map(float, [gt_x, gt_y, gt_w, gt_h])

            # Compute intersection over union (IOU)
            x1 = max(gt_x - gt_w/2, pred_x - pred_w/2)
            y1 = max(gt_y - gt_h/2, pred_y - pred_h/2)
            x2 = min(gt_x + gt_w/2, pred_x + pred_w/2)
            y2 = min(gt_y + gt_h/2, pred_y + pred_h/2)
            intersection = max(0, x2 - x1) * max(0, y2 - y1)
            union = gt_w * gt_h + pred_w * pred_h - intersection
            iou = intersection / union

            # Update best matching ground truth object if necessary
            if iou > best_iou:
                best_iou = iou
                best_gt_idx = gt_idx
        
        # If IOU is above threshold and ground truth object has not been matched yet, count as true positive
        if best_iou >= iou_threshold and not tp[best_gt_idx]:
            if pred_conf >= confidence_threshold:
                tp[best_gt_idx] = True
            else:
                fp[pred_idx] = True

    # Add accumulated true positives and false positives to overall list
    true_positives += tp
    false_positives += fp

# Compute precision, recall, and AP
num_predictions = len(true_positives)
cumulative_true_positives = [sum(true_positives[:i+1]) for i in range(num_predictions)]
cumulative_false_positives = [sum(false_positives[:i+1]) for i in range(num_predictions)]

precision = [0] * num_predictions
recall = [0] * num_predictions
ap = 0
for i in range(num_predictions):
    precision[i] = cumulative_true_positives[i] / (np.finfo(float).eps + cumulative_true_positives[i] + cumulative_false_positives[i])
    recall[i] = cumulative_true_positives[i] / total_gt_objects
    if recall[i] >= recall[i-1]:
        ap += (recall[i] - recall[i-1]) * precision[i]

print("Recall:", recall[-1])
print("Precision:", precision[-1])
print("mAP:", ap)

### Train on bounding boxes

In [ ]:
!python yolov5/train.py --data frogs-1/data.yaml --img 320 --epochs 200 --batch 128 --name trained-od

### Computes metrics

In [ ]:
import os

preds_folder_od = "yolov5/runs/detect/exp/labels/"

# Get list of file names in gt folder
gt_files = os.listdir(gt_folder)

# Loop through each file in gt folder
for file_name in gt_files:
    # Check if corresponding file exists in preds folder
    preds_file_path = os.path.join(preds_folder_od, file_name)
    if not os.path.exists(preds_file_path):
        # Create file in preds folder with default values
        with open(preds_file_path, "w") as f:
            f.write("0 0 0 0 0 -1")

In [ ]:
import os

# Set parameters for computing AP
confidence_threshold = 0
iou_threshold = 0

# Initialize variables for computing precision, recall, and AP
true_positives = []
false_positives = []
total_gt_objects = 0
c = 0

# Loop through each file in gt folder
for file_name in os.listdir(gt_folder):
    c += 1
    # Load ground truth data
    gt_file_path = os.path.join(gt_folder, file_name)
    with open(gt_file_path, "r") as f:
        gt_data = f.readlines()
    gt_data = [line.strip().split() for line in gt_data]

    # Load prediction data
    preds_file_path = os.path.join(preds_folder, file_name)
    with open(preds_file_path, "r") as f:
        preds_data = f.readlines()
    preds_data = [line.strip().split() for line in preds_data]

    # Initialize variables for computing recall and AP for this file
    num_gt_objects = len(gt_data)
    total_gt_objects += num_gt_objects
    tp = [False] * num_gt_objects
    fp = [False] * len(preds_data)
    
    # Loop through each prediction
    for pred_idx, pred in enumerate(preds_data):
        pred_x, pred_y, pred_w, pred_h, pred_conf, pred_class = pred
        if pred_class == '-1':
            pass
        pred_x, pred_y, pred_w, pred_h, pred_conf = map(float, [pred_x, pred_y, pred_w, pred_h, pred_conf])
        pred_conf = float(pred_conf)

        # Find best matching ground truth object
        best_iou = 0
        best_gt_idx = -1
        for gt_idx, gt in enumerate(gt_data):
            gt_class, gt_x, gt_y, gt_w, gt_h = gt
            gt_x, gt_y, gt_w, gt_h = map(float, [gt_x, gt_y, gt_w, gt_h])

            # Compute intersection over union (IOU)
            x1 = max(gt_x - gt_w/2, pred_x - pred_w/2)
            y1 = max(gt_y - gt_h/2, pred_y - pred_h/2)
            x2 = min(gt_x + gt_w/2, pred_x + pred_w/2)
            y2 = min(gt_y + gt_h/2, pred_y + pred_h/2)
            intersection = max(0, x2 - x1) * max(0, y2 - y1)
            union = gt_w * gt_h + pred_w * pred_h - intersection
            iou = intersection / union

            # Update best matching ground truth object if necessary
            if iou > best_iou:
                best_iou = iou
                best_gt_idx = gt_idx
        
        # If IOU is above threshold and ground truth object has not been matched yet, count as true positive
        if best_iou >= iou_threshold and not tp[best_gt_idx]:
            if pred_conf >= confidence_threshold:
                tp[best_gt_idx] = True
            else:
                fp[pred_idx] = True

    # Add accumulated true positives and false positives to overall list
    true_positives += tp
    false_positives += fp

# Compute precision, recall, and AP
num_predictions = len(true_positives)
cumulative_true_positives = [sum(true_positives[:i+1]) for i in range(num_predictions)]
cumulative_false_positives = [sum(false_positives[:i+1]) for i in range(num_predictions)]

precision = [0] * num_predictions
recall = [0] * num_predictions
ap = 0
for i in range(num_predictions):
    precision[i] = cumulative_true_positives[i] / (np.finfo(float).eps + cumulative_true_positives[i] + cumulative_false_positives[i])
    recall[i] = cumulative_true_positives[i] / total_gt_objects
    if recall[i] >= recall[i-1]:
        ap += (recall[i] - recall[i-1]) * precision[i]

print("Recall:", recall[-1])
print("Precision:", precision[-1])
print("mAP:", ap)